## $\ast$ MACHO VS LC [TAP+]

In [1]:
from time import strftime
import os

from astroquery.utils.tap.core import TapPlus

macho_url = "http://machotap.asvo.nci.org.au/ncitap/tap"
tap=TapPlus(url=macho_url)
tables = tap.load_tables()

print('#tables in TAP macho : ', len(tables))
for ii in range(len(tables)):
    table = tables[ii]
    print('\nIndice ', ii, '\n', table)

Created TAP+ (v1.0.1) - Connection:
	Host: machotap.asvo.nci.org.au
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Retrieving tables...
Parsing tables...
Done.
#tables in TAP macho :  14

Indice  0 
 TAP Table name: public.public.fieldtile_view
Description: None
Num. columns: 2

Indice  1 
 TAP Table name: public.public.field_view
Description: None
Num. columns: 5

Indice  2 
 TAP Table name: public.public.observation_view
Description: None
Num. columns: 22

Indice  3 
 TAP Table name: public.public.photometry_radec_view
Description: None
Num. columns: 40

Indice  4 
 TAP Table name: public.public.photometry_view
Description: None
Num. columns: 38

Indice  5 
 TAP Table name: public.public.starstat_view
Description: None
Num. columns: 72

Indice  6 
 TAP Table name: public.public.star_view
Description: None
Num. columns: 6

Indice  7 
 TAP Table name: public.public.tile_view
Description: None
Num. columns: 4

Indice  8 
 TAP Table name: public.public.varstar_view
Description: None
Num. col

In [ ]:
tabphotomPos = tables[3]     # 3, photometry_radec_view
tabphotom    = tables[4]     # 4, photometry_view
tabvarstar   = tables[8]     # 8, varstar_view

'''
    >> Columns ID for public.photometry_view [MACHO TAP] :  
fieldid ; rmag ; ramp ; bmag ; bamp ; 
tileid ; rerr ; rxpix ; berr ; bxpix ; 
seqn ; rnormsky ; rypix ; bnormsky ; bypix ; 
obsid ; rtype ; ravesky ; btype ; bavesky ; 
dateobs ; rcrowd ; rfwhm ; bcrowd ; bfwhm ; 
sideofpier ; rchi2 ; rtobs ; bchi2 ; btobs ; 
exposure ; rmpix ; rcut ; bmpix ; bcut ; 
airmass ; rcosmicrf ; " ; bcosmicrf ; " ; 

    >> Columns ID for public.varstar_view [MACHO TAP]:  
starid ; period_reqb ; magave_r ; amp_r ; suprsa_r ; varindex
rarad ; period_r ; magave_b ; amp_b ; suprsa_b ; classification
decrad ; period_b ; magave_bminusr ; amp_bminusr ; chi2r_r ; points_total
field  ; magave_kv ; amp_reqb ; chi2r_b ; points_concurrent
tile  ; magave_kr ;  ; sig_r ; points_r
seqn  ; magave_kvminuskr ;  ; sig_b ; points_b
'''

In [5]:
query_1 = "SELECT \
            * \
            FROM public.varstar_view \
            ORDER BY public.varstar_view.field ASC"
job=tap.launch_job_async(query_1,
                         dump_to_file=True, 
                         output_file='data/vs_catalog.csv', 
                         output_format='csv')
print(job)                      
resu_vs=job.get_results()
resu_vs.pprint()

Jobid: 1534455274458A
Phase: COMPLETED
Owner: None
Output file: data/vs_catalog.csv
Results: None
 starid   rarad   decrad  field ... points_concurrent points_r points_b
-------- ------- -------- ----- ... ----------------- -------- --------
44704475 1.34813 -1.20238     1 ...               450      458     1095
44704124 1.34764 -1.20135     1 ...               455      466     1107
44703767 1.34838 -1.20167     1 ...               205      210      774
44703441 1.34979 -1.20188     1 ...                23       23      748
44702889 1.34767 -1.20189     1 ...               464      471     1122
44709949 1.34931  -1.1995     1 ...                41       41      514
44709252 1.34967 -1.19968     1 ...                24       24      915
44709230  1.3477 -1.19992     1 ...               443      456     1057
44708896 1.34943 -1.19963     1 ...               327      339     1028
44708687 1.34945 -1.19975     1 ...               327      340     1040
     ...     ...      ...   ... ...   

In [ ]:
%%time
nb_vs = len(resu_vs['starid'])
directo = 'VS/LC/' #  later renamed to : 'VS/retrieved_21474LCs/'

selec_format = 'csv'
selec_formatExt = '.csv'   #'.txt'

indices_ = range(nb_vs)

logfilename = strftime('VS/logs/ipy_log_machoTAP_%Y-%m-%d')+'_nbRound'+nbRound+".log" 
logfilepath = "%s/%s" % (os.getcwd(),logfilename) 

file_handle = open(logfilepath,'w') 
file_handle.write('#######################################################################\n')  
file_handle.write('# Log : '+ strftime('%Y-%m-%d %H:%M:%S\n')) 
file_handle.write('#######################################################################\n') 

for id_vs in indices_:
    sel_field = str(resu_vs['field'][id_vs])
    sel_tile  = str(resu_vs['tile'][id_vs])
    sel_seqn  = str(resu_vs['seqn'][id_vs])
    query_   = "SELECT \
                    tab_lc.fieldid , tab_lc.tileid , tab_lc.seqn, tab_lc.obsid ,\
                    tab_lc.dateobs , tab_lc.exposure , tab_lc.airmass ,\
                    tab_lc.rmag , tab_lc.rerr , tab_lc.rnormsky , tab_lc.rtype ,\
                    tab_lc.rcrowd, tab_lc.rchi2 , tab_lc.rmpix , tab_lc.rcosmicrf ,\
                    tab_lc.ramp , tab_lc.rxpix, tab_lc.rypix , tab_lc.ravesky ,\
                    tab_lc.rfwhm , tab_lc.rtobs , tab_lc.rcut ,\
                    tab_lc.bmag, tab_lc.berr , tab_lc.bnormsky , tab_lc.btype ,\
                    tab_lc.bcrowd , tab_lc.bchi2 , tab_lc.bmpix , tab_lc.bcosmicrf ,\
                    tab_lc.bamp , tab_lc.bxpix , tab_lc.bypix, tab_lc.bavesky ,\
                    tab_lc.bfwhm, tab_lc.btobs , tab_lc.bcut \
                    FROM public.varstar_view AS tab_vs \
                    INNER JOIN public.photometry_view AS tab_lc \
                    ON (tab_lc.fieldid=tab_vs.field \
                        AND tab_lc.tileid=tab_vs.tile \
                        AND tab_lc.seqn=tab_vs.seqn) \
                    WHERE (tab_lc.fieldid="+sel_field+" \
                        AND tab_lc.tileid="+sel_tile+" \
                        AND tab_lc.seqn="+sel_seqn+")"
    
    selected_name = sel_field+'.'+sel_tile+'.'+sel_seqn
    
    #job=tap.launch_job_async(query_, 
    job = tap.launch_job(query_,
                         dump_to_file=True,
                         output_file=directo+selected_name+selec_formatExt, 
                         output_format=selec_format,
                         verbose=False)

    resu = job.get_results()
    
    file_handle.write(">>>>>> \tTAP+ QUERY FOR VS (field.tile.seqn): "+selected_name+'\n')
    file_handle.write(">>>>>> \tRESULT [#LC_datapoints "+str(len(resu))+']\n')
    file_handle.write(">>>>>> \tCOMPLETED JOB:\n"+ str(job)+'\n')
    file_handle.write('----------------------------------------------------------------------\n')
                                
file_handle.write('>>>>>> Finished loading VS LCs from MACHO photometric database ')

file_handle.close() 


In [ ]:
##
#7.8028.2713     (?, among 311 empty tables from TAP+ query)
selF='7'
selT='8028'
selS='2713'

query_8="SELECT \
            tab_lc.fieldid , tab_lc.tileid , tab_lc.seqn, tab_lc.obsid ,\
            tab_lc.dateobs , tab_lc.exposure , tab_lc.airmass ,\
            tab_lc.rmag , tab_lc.rerr , tab_lc.rnormsky , tab_lc.rtype ,\
            tab_lc.rcrowd, tab_lc.rchi2 , tab_lc.rmpix , tab_lc.rcosmicrf ,\
            tab_lc.ramp , tab_lc.rxpix, tab_lc.rypix , tab_lc.ravesky ,\
            tab_lc.rfwhm , tab_lc.rtobs , tab_lc.rcut ,\
            tab_lc.bmag, tab_lc.berr , tab_lc.bnormsky , tab_lc.btype ,\
            tab_lc.bcrowd , tab_lc.bchi2 , tab_lc.bmpix , tab_lc.bcosmicrf ,\
            tab_lc.bamp , tab_lc.bxpix , tab_lc.bypix, tab_lc.bavesky ,\
            tab_lc.bfwhm, tab_lc.btobs , tab_lc.bcut \
            FROM public.varstar_view AS tab_vs \
            INNER JOIN public.photometry_view AS tab_lc \
            ON (tab_lc.fieldid=tab_vs.field \
                AND tab_lc.tileid=tab_vs.tile \
                AND tab_lc.seqn=tab_vs.seqn) \
            WHERE (tab_lc.fieldid="+selF+" \
                AND tab_lc.tileid="+selT+"  \
                AND tab_lc.seqn="+selS+" )"

query_ = query_8
id_selected = 'query8'

directo = 'VS/tempo_test/'    #folder to create
selected = directo+'test_'+id_selected+'_testos.csv'

#job=tap.launch_job_async(query_, 
job=tap.launch_job(query_, 
                   dump_to_file=True, 
                   output_file=selected, 
                   output_format='csv',
                   verbose = True)

#print(job)
resu=job.get_results()
resu.pprint()

In [ ]:
'''
import pyvo as vo
service = vo.dal.TAPService(macho_url)
tables = service.tables

nkeys = len([i for i in tables.keys()])


##  public.fieldtile_view
##  public.field_view
##  public.observation_view
##  public.photometry_radec_view
##  public.photometry_view
##  public.starstat_view
##  public.star_view
##  public.tile_view
##  public.varstar_view
##  TAP_SCHEMA.columns
##  TAP_SCHEMA.key_columns
##  TAP_SCHEMA.keys
##  TAP_SCHEMA.schemas
##  TAP_SCHEMA.tables


print('#tables in TAP macho : ', nkeys)
for i in tables.keys():
    table = tables[i]
    print(f'\t {i} \t ncolumns={len(table.columns)}')
    
    
tabphotomPos = tables['public.photometry_radec_view']     # 3, photometry_radec_view
tabphotom    = tables['public.photometry_view']     # 4, photometry_view
tabvarstar   = tables['public.varstar_view']     # 8, varstar_view



[i.name for i in tabphotomPos.columns]
'''